In [1]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import nltk
from nltk.corpus import stopwords
from keras import Sequential
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow_hub as hub
import tensorflow as tf

2023-10-30 09:53:25.047894: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dataset=pd.read_json('corpus/sarcasm.json')

In [3]:
dataset['is_sarcastic'].value_counts()


is_sarcastic
0    14985
1    11724
Name: count, dtype: int64

In [4]:
sentences=dataset['headline'].to_list()
label=dataset['is_sarcastic'].to_list()


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mansoor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stopwrods=stopwords.words('english')

In [7]:
stopwrods

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [8]:
train_sent=sentences[:20000]
valid_sent=sentences[20000:]

train_lab=label[:20000]
valied_lab=label[20000:]


In [9]:
tokenizer=Tokenizer(num_words=10000,oov_token='<OOV>')
tokenizer.fit_on_texts(train_sent)

In [10]:
word_index=tokenizer.word_index

In [11]:
train_seq=tokenizer.texts_to_sequences(train_sent)
valied_seq=tokenizer.texts_to_sequences(valid_sent)

train_padded=pad_sequences(train_seq,padding='post',maxlen=34)
valied_padded=pad_sequences(valied_seq,padding='post',maxlen=34)


In [12]:

hub_layer = hub.KerasLayer(
    "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1",
    output_shape=[20], input_shape=[],
    dtype=tf.string, trainable=False
)

HTTPError: HTTP Error 403: Forbidden

In [ ]:
model=Sequential([
    layers.Embedding(10000,64),
    layers.Bidirectional(layers.LSTM(64),input_shape=(20000,34,1)),
    layers.Dense(24,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

In [ ]:

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history=model.fit(train_sent,train_lab,epochs=10,batch_size=33,validation_data=(valid_sent,valied_lab))

In [ ]:
train_loss=history.history['loss']
valied_loss=history.history['val_loss']
epochs = range(1,16)
plt.plot(epochs, train_loss, 'g', label='Training loss')
plt.plot(epochs, valied_loss, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_loss=history.history['accuracy']
valied_loss=history.history['val_accuracy']
epochs = range(1,16)
plt.plot(epochs, train_loss, 'g', label='Training loss')
plt.plot(epochs, valied_loss, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
from transformers  import pipeline

In [ ]:
my_pip=pipeline(task='sentiment-analysis')